In [6]:
df = pd.read_csv('after_data_engineering.csv')
pd.set_option('display.max_columns', None)
df

,transaction_id,pickup_city,delivery_city,pickup_lat,pickup_lon,delivery_lat,delivery_lon,distance_km,weight_kg,volume_m3,pickup_time,estimated_delivery,actual_delivery,actual_delivery_time_hours,earliest_delivery,latest_delivery,status,priority,vehicle_type,vehicle_capacity_weight,vehicle_capacity_volume,payment_method,price_inr,traffic_multiplier,is_monsoon,weather_main,temperature,traffic_level_category,delivery_time_diff,weight_utilization(%),volume_utilization(%)
0,IN000003,Chennai,Kolkata,0.008637,0.479216,0.611050,0.995456,1389.21,307.60,2.006,2024-02-07 01:23:31.227104,2024-03-13 14:03:27.442155,2024-03-13 13:49:56.476180,852.440347,2024-02-07 09:23:31.227104,2024-02-08 01:23:31.227104,Delivered,Next Day,Mahindra Pickup,1200,12.0,Digital Wallet,31349.20,1.23,False,Haze,26.03,High,-0.225268,25.633333,16.716667
1,IN000005,Kolkata,Mumbai,0.606553,0.993964,0.387256,0.010690,1754.85,545.75,1.599,2024-02-07 01:26:26.139326,2024-04-07 07:30:11.409007,2024-04-07 07:53:48.267264,1446.456147,2024-02-07 02:26:26.139326,2024-02-07 05:26:26.139326,Delivered,Same Day,Mahindra Pickup,1200,12.0,Credit Card,63880.40,1.65,False,Smoke,29.93,High,0.393572,45.479167,13.325000
2,IN000006,Delhi,Kolkata,0.992609,0.282752,0.618409,0.994432,1402.98,27.19,0.035,2024-02-07 01:28:15.781805,2024-03-27 03:36:44.183867,2024-03-27 03:29:53.908525,1178.027257,2024-02-07 09:28:15.781805,2024-02-08 01:28:15.781805,Delivered,Next Day,Delivery Bike,50,0.3,Credit Card,27716.06,1.68,False,Haze,26.03,High,-0.113965,54.380000,11.666667
3,IN000008,Bangalore,Kolkata,0.005602,0.301448,0.613072,0.987582,1602.52,30.77,0.194,2024-02-07 01:31:13.116943,2024-03-22 09:54:27.943414,2024-03-22 13:30:14.249860,1067.983648,2024-02-07 02:31:13.116943,2024-02-07 04:31:13.116943,Delivered,Premium,Delivery Bike,50,0.3,Net Banking,60912.21,1.33,False,Clouds,25.99,High,3.596196,61.540000,64.666667
4,IN000009,Mumbai,Chennai,0.391706,0.008794,0.012644,0.482534,1060.51,36.23,3.629,2024-02-07 01:37:58.197176,2024-03-13 23:39:05.397500,2024-03-14 00:08:28.770956,862.508493,2024-02-08 01:37:58.197176,2024-02-09 01:37:58.197176,Delivered,Standard,Cargo Auto,500,5.0,UPI,16632.78,1.63,False,Clouds,28.90,High,0.489826,7.246000,72.580000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35947,IN059994,Mumbai,Pune,0.391821,0.004544,0.355441,0.067921,127.59,305.11,0.091,2024-12-03 00:21:24.437174,2024-12-07 23:50:07.176110,2024-12-07 23:37:14.267319,119.263842,2024-12-03 01:21:24.437174,2024-12-03 03:21:24.437174,Delivered,Premium,Tata Ace,750,8.0,Credit Card,12435.17,1.87,False,Clouds,30.92,High,-0.214697,40.681333,1.137500
35948,IN059995,Pune,Hyderabad,0.351938,0.065564,0.282074,0.365660,536.77,468.64,0.326,2024-12-03 00:21:59.049294,2024-12-16 07:30:10.188551,2024-12-16 07:20:41.317120,318.978408,2024-12-04 00:21:59.049294,2024-12-05 00:21:59.049294,Delivered,Standard,Tata Ace,750,8.0,Credit Card,12770.54,1.19,False,Clouds,29.38,High,-0.158020,62.485333,4.075000
35949,IN059996,Delhi,Pune,0.995218,0.279219,0.361628,0.068381,1173.05,80.73,5.319,2024-12-03 00:28:30.413235,2025-01-07 16:13:54.901902,2025-01-07 16:29:51.159444,856.022430,2024-12-03 01:28:30.413235,2024-12-03 03:28:30.413235,Delivered,Premium,Tata Ace,750,8.0,Net Banking,47337.25,1.46,False,Clouds,31.41,High,0.265627,10.764000,66.487500
35950,IN059999,Hyderabad,Pune,0.286271,0.369141,0.358209,0.063938,546.20,227.59,3.429,2024-12-03 00:50:21.052310,2024-12-18 10:27:28.379421,2024-12-18 10:07:49.215546,369.291156,2024-12-04 00:50:21.052310,2024-12-05 00:50:21.052310,Delivered,Standard,Cargo Auto,500,5.0,Debit Card,10811.83,1.35,False,Clouds,31.41,High,-0.327546,45.518000,68.580000


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from datetime import datetime

# Load and preprocess the data
def prepare_data(df):
    # Extract relevant features
    features = {
        'numeric_features': [
            'distance_km',
            'weight_kg',
            'volume_m3',
            'traffic_multiplier',
            'temperature'
        ],
        'categorical_features': [
            'pickup_city',
            'delivery_city',
            'priority',
            'vehicle_type',
            'weather_main',
            'traffic_level_category'
        ]
    }
    
    return df, features

# Create the model pipeline
def create_model_pipeline(numeric_features, categorical_features):
    # Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
        ])
    
    # Create pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            n_jobs=-1
        ))
    ])
    
    return pipeline

def evaluate_model(pipeline, X, y, dataset_name=""):
    """
    Evaluate model performance using multiple metrics
    """
    # Make predictions
    y_pred = pipeline.predict(X)
    
    # Calculate metrics
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y, y_pred)
    
    print(f"\n{dataset_name} Metrics:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    return mse, rmse, r2

def main():
    # Read the data
    df = pd.read_csv('after_data_engineering.csv')
    
    # Prepare the data
    df, features = prepare_data(df)
    
    # Split features and target
    X = df[features['numeric_features'] + features['categorical_features']]
    y = df['actual_delivery_time_hours']
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Create and train the pipeline
    pipeline = create_model_pipeline(
        features['numeric_features'],
        features['categorical_features']
    )
    
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # Evaluate model on both training and test sets
    train_mse, train_rmse, train_r2 = evaluate_model(pipeline, X_train, y_train, "Training")
    test_mse, test_rmse, test_r2 = evaluate_model(pipeline, X_test, y_test, "Test")
    
    # # Calculate feature importance
    # feature_names = (features['numeric_features'] + 
    #                 pipeline.named_steps['preprocessor']
    #                 .named_transformers_['cat']
    #                 .get_feature_names_out(features['categorical_features']).tolist())
    
    # importances = pipeline.named_steps['regressor'].feature_importances_
    
    # # Create feature importance DataFrame
    # importance_df = pd.DataFrame({
    #     'feature': feature_names,
    #     'importance': importances
    # }).sort_values('importance', ascending=False)
    
    # print("\nTop 10 Most Important Features:")
    # print(importance_df.head(10))
    
    # Save the model
    joblib.dump(pipeline, 'delivery_time_prediction_model.pkl')
    
    return pipeline, features

if __name__ == "__main__":
    main()


Training Metrics:
MSE: 12.7590
RMSE: 3.5720
R² Score: 1.0000

Test Metrics:
MSE: 40.9658
RMSE: 6.4005
R² Score: 0.9999
